# wobble $\longleftrightarrow$ WC reaction in codon-anticodon decoding 

This notebook interactively solves the equations for the error rate. See the accompanying preprint/publication for derivations and detailed explanation. Nomenclature of the decoding states and rate constants is taken from [Pavlov and Ehrenberg, 2018] (doi:10.1146/annurev-biophys-060414-034148)

In the classical Michaelis-Menten formulation, the error rate in decoding in translation:

\begin{equation}
\eta_0 = \frac{R^{nc}}{ R^{c}} = \frac{(k_{cat}/K_{m})^{nc}}{(k_{cat}/K_{m})^{c}} = \frac{k_{4}^{nc}[C_{4}^{nc}]}{k_{4}^{c}[C_{4}^{c}]}
\end{equation}

where $R^{i}$ is the rate of decoding, $k_{cat}^{i}$ is the catalytic rate constant, $K_m^{i}$ is the Michaelis-Menten constant, $[C_{4}^{i}]$ is the steady-state concentration of C4 state, and  $k_{4}^{i}$ is the rate constant of GTPase activation, for $i = c$ (cognate), $nc$ (near-cognate)

In our model, the error rate induced by the wb-WC reaction:

\begin{equation}
\eta  = \frac{[C_{4}^{nc}]}{[C_{4}^{c}]}(P_{wc}^{eq} + (P_{wc}^{C3} \frac{q_{4}^{nc}}{k_{4}^{c} + q_{4}^{c}} - P_{wc}^{eq}) \exp{(-\frac{k_f + k_r}{k_{4}^{c} + q_{4}^{c}})})
\end{equation}

where $k_f$, $k_r$ and $P_{wc}^{eq}$ are the forward and reverse rate constants of the wb-WC reaction in $C4$, and the corresponding equilbrium WC (product) population, respectively; $P_{wc}^{C3}$ -- equilibrium WC population in state $C3$. 

In the plot below, you can interactively change these three parameters (in terms of free energy differences).

In [4]:
## modules
import numpy as np
import matplotlib
import ipympl
import matplotlib.pyplot as plt
import ipywidgets as widgets
%matplotlib widget
############################
### GLOBAL CONSTANTS #######
############################
temperature = 298.0               ## [K]
Rconst = 1.9858775e-3             ## [kcal / (mol * K)]
RT = temperature * Rconst
kbc = 1.38064852e-23                     ## Boltzmann constant (scipy had some problems in Binder, so here is manual entry)
hpl = 6.62607004e-34                     ## Planck constant
############################
### DECODING CONSTANTS #####
############################
## all but k2 and q3c/nc are from Rodnina 2017 (http://dx.doi.org/10.1098/rstb.2016.0182)
## k2 and q3c/nc : Ehrenberg 2018 (10.1146/annurev-biophys-060414-034148)
all_kfs = {
    'k1': 1.4e8,
    'q2': 85,
    'k2': 7.2e2,
    'q3c': 2.5e1,
    'q3nc': 3.9e3,
    'k3': 180,
    'q4c': 0.2,
    'q4nc': 140,
    'k4c': 190,
    'k4nc': 0.6,
    'k4wb': 0
    }
############################
###      FUNCTIONS     #####
############################
def calc_kfkr(dg, ddg):
    """ calculates rate constants from dG and ddG"""
    kf = (kbc * temperature/hpl) * np.exp(-ddg/(RT))
    kr = (kbc * temperature/hpl) * np.exp(-(ddg - dg)/(RT))
    return kf, kr
###########
def calc_pwc_t(kf, kr, t, wc0 = 0):
    """ equation for product concentration 
    in a first-order reversible reaction"""
    wceq = (kf/kr) / (kf/kr + 1)
    pwc = wceq + (wc0 - wceq) * np.exp(-(kf + kr) * t)
    return pwc  
###########
def calc_pwc_eq(kf, kr):
    """ calculates equlibrium population of the product"""
    wceq = (kf/kr) / (kf/kr + 1)
    return wceq
############
############
def calc_eta(kdict0, kfkr, c3_corr = 'yes', c3_params = [3.4, 6], change = 'none'):
    """ calculates error rate from the equations in the notebook
    
    Arguments:
    kdict0: dictionary with decoding rate constants;
    kfkr: forward and reverse rate constants of the wb-WC reaction in C4 [k_f, k_r]; for a classical system, type any string
    
    Parameters:
    c3_corr: whether C3 population is included in the error rate;
    c3_params: [dG_C3,ddG_c3] of the wb-WC reaction in C3;
    change: if not str, expects 2D list [[k_i, value], [k_j, value], ..., [k_n, value]]
    
    Returns:
    eta: error rate
    
    If both k4c and k4nc are supplied, the function calculates the classical error tate eta0;
    if not, k4nc is rescaled to introduce Pwc
    
    """
    kdict = kdict0.copy()   
    ## change rates
    if type(change) != str:
        for i in change:
            kdict[i[0]] = i[1]
    ##########
    tau  = 1/(kdict['k4c'] + kdict['q4c'])    ### residence time
    ##########
    if type(kfkr) != str:
        if c3_corr == 'no':
            pwc_c3_part = 0
            pwc = calc_pwc_t(kfkr[0], kfkr[1], tau, wc0 = 0)
        else:
            c3_kfkr = calc_kfkr(c3_params[0], c3_params[1])
            pwc_c3_part = calc_pwc_eq(c3_kfkr[0], c3_kfkr[1]) * (kdict['q4nc']) / (kdict['q4c'] + kdict['k4c'])
            pwc = calc_pwc_t(kfkr[0], kfkr[1], tau, wc0 = pwc_c3_part)
        ### must substitute k4_nc = k4_c * pwc
        if ('k4nc' not in [i[0] for i in change]):
            kdict['k4nc'] = kdict['k4c'] * pwc
    #######
    ## the rates are calculated from discard paremeters ( Pavlov and Ehrenberg, 2018 (10.1146/annurev-biophys-060414-034148))
    a2c, a2nc = (kdict['q2'] / kdict['k2']), (kdict['q2'] / kdict['k2'])
    a3c, a3nc = (kdict['q3c'] / kdict['k3']), (kdict['q3nc'] / kdict['k3'])
    a4c, a4nc = (kdict['q4c'] / kdict['k4c']), (kdict['q4nc'] / kdict['k4nc'])
    ####
    rate_c = kdict['k1'] / (1 + a2c * (1 + a3c * (1 + a4c)))
    rate_nc = kdict['k1'] / (1 + a2nc * (1 + a3nc * (1 + a4nc)))
    ###
    eta = rate_nc / rate_c                          ## fow wb-WC, already includes Pwc, as k4_nc was rescaled; for classical, pwc = k4nc/k4c
    ##
    return eta
############################
###      PLOTTING      #####
############################
#plt.style.use('seaborn-notebook')
k_range = np.logspace(-5, 6, 100)
eta0_k, eta0_q = [], []
q4_coeff = all_kfs['q4nc']/all_kfs['q4c'] ## 
###

### classical error rate
for i in k_range:
    eta0_k.append(calc_eta(all_kfs, 'class', change = [['k4c', i], ['k4nc', i * all_kfs['k4nc']/all_kfs['k4c']]]))
    eta0_q.append(calc_eta(all_kfs, 'class', change = [['q4c', i], ['q4nc', i * q4_coeff]]))
#############
#############
## FIGURE ###
#############
fig = plt.figure(1,figsize=(9, 4))
ax1, ax2 = fig.add_subplot(1,2,1), fig.add_subplot(1,2,2)
for ax in ax1,ax2:
    ax.set_yscale('log')
    ax.set_xscale('log')
    ax.grid(alpha = 0.3, ls = 'dashed')
    ax.set_ylim(1e-8, 1)
    ax.set_xlim(k_range[0], k_range[-1])
    
###
ax1.set_ylabel(r'$\eta$',fontsize=11, labelpad=1.3)
##
ax1.set_xlabel('$k_4^{c}, s^{-1}$',fontsize=11, labelpad=1.3)
ax2.set_xlabel('$q_4^{c}, s^{-1}$',fontsize=11, labelpad=1.3)
ax2.yaxis.set_ticks_position('right')
## q4nc axis
ax3 = ax2.twiny()
# #ax3.set_axis_off()
ax3.set_xscale('log')
# ax3.tick_params(pad=-1.2)
ax3.set_xlabel('$q_4^{nc}, s^{-1}$',fontsize=11, labelpad=8.3)
ax3.set_xlim(k_range[0]*q4_coeff, k_range[-1]*q4_coeff)
# ax3.set_xlim(q4_range[0]*q4nc0_coeff, q4_range[-1]*q4nc0_coeff)
ax3.axvline(all_kfs['q4nc'], ls = 'dashed', lw = 1.0, color = 'gray', alpha = 0.8)
##
ax1.plot(k_range, eta0_k, color = 'gray', zorder = 100, lw = 3.5, ls = 'solid', alpha = 1.0, label = r'$\eta _0$')
ax2.plot(k_range, eta0_q, color = 'gray', zorder = 100, lw = 3.5, ls = 'solid', alpha = 1.0, label = r'$\eta _0$')
##
ax1.axvline(all_kfs['k4c'], ls = 'dashed', lw = 1.0, color = 'gray', alpha = 0.8)
ax2.axvline(all_kfs['q4c'], ls = 'dashed', lw = 1.0, color = 'gray', alpha = 0.8)
##
#plt.legend(fontsize = 6)
plt.subplots_adjust(left=0.08,right=0.95,bottom=0.14,top=0.85,wspace=0.02,hspace=0.27)
###########################
## INTERACTIVE REACTION ###
###########################
def plot_sel_rates(sel_dG, sel_ddG, C3_dG):
    sel_rates = calc_kfkr(sel_dG, sel_ddG)
    eta_k, eta_q = [], []
    ## remove the interactive line from the plot
    while len(ax1.lines) > 2:
        ax1.lines[-1].remove()
        ax2.lines[-1].remove()
    ###
    for i in k_range:
        eta_k.append(calc_eta(all_kfs, sel_rates, c3_params = [C3_dG, 6], change = [['k4c', i]]))
        eta_q.append(calc_eta(all_kfs, sel_rates, c3_params = [C3_dG, 6], change = [['q4c', i], ['q4nc', i * q4_coeff]]))
    ###
    k_curve = ax1.plot(k_range, eta_k, color = 'black', zorder = 105, lw = 1.8, ls = 'solid')
    q_curve = ax2.plot(k_range, eta_q, color = 'black', zorder = 105, lw = 1.8, ls = 'solid')
###########
widgets.interact(plot_sel_rates, 
                 sel_dG = widgets.FloatSlider(min = -3, max = 8, step = 0.25, value = -1.0, description = '$\Delta G_{C4}$, kcal/mol'), 
                 sel_ddG = widgets.FloatSlider(min = 4, max = 21, step = 0.25, value = 17.8, description = '$\Delta G_{TS}$, kcal/mol'), 
                 C3_dG = widgets.FloatSlider(min = 0, max = 10, step = 0.25, value = 3.4, description = '$\Delta G_{C3}$, kcal/mol'))


AttributeError: module 'scipy' has no attribute 'constants'

In [6]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});

<IPython.core.display.Javascript object>

In [5]:
%%javascript
MathJax.Hub.Queue(
  ["resetEquationNumbers", MathJax.InputJax.TeX],
  ["PreProcess", MathJax.Hub],
  ["Reprocess", MathJax.Hub]
);


<IPython.core.display.Javascript object>